<a href="https://colab.research.google.com/github/Jorn1977/rainfall/blob/master/RainfallDashboard30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
%pip install dash dash-core-components dash-html-components
%pip install dash dash-core-components dash-html-components plotly
%pip install dash-bootstrap-components
%pip install --upgrade pip

In [15]:
from dash import Dash, dcc, html, Input, Output, State
import requests
from bs4 import BeautifulSoup
import plotly.express as px
import pandas as pd
from datetime import datetime, timedelta
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import urllib.parse
import numpy as np # Import numpy

# Fetch the initial list of site names and coordinates
url = "https://hilltopserver.horizons.govt.nz/data.hts?Service=Hilltop&Request=SiteList&Location=LatLong"
response = requests.get(url)
html_content = response.text
soup = BeautifulSoup(html_content, 'xml')

site_names = []
for site in soup.find_all('Site'):
    lat_tag = site.find('Latitude')
    long_tag = site.find('Longitude')
    if lat_tag and long_tag:
        site_names.append([
            site.get('Name'),
            float(lat_tag.text),
            float(long_tag.text)
        ])

df_sites = pd.DataFrame(site_names, columns=['Site', 'Latitude', 'Longitude'])

# Fetch the initial list of rainfall site names
url1 = "https://hilltopserver.horizons.govt.nz/data.hts?Service=Hilltop&Request=SiteList&Collection=rainfall"
response1 = requests.get(url1)
html_content1 = response1.text
soup1 = BeautifulSoup(html_content1, 'xml')

site_names1 = []
for site1 in soup1.find_all('Site'):
        site_names1.append([
            site1.get('Name')
        ])
df_rfsites = pd.DataFrame(site_names1, columns=['Site'])

merged_df = pd.merge(df_sites, df_rfsites, on='Site')

fig = px.scatter_map(merged_df, lat="Latitude", lon="Longitude", hover_name="Site",
                     hover_data={"Latitude": False, "Longitude": False},  # Explicitly set hover_data to an empty list
                     color_discrete_sequence=["blue"], zoom=6, height=600)

fig.update_traces(marker=dict(size=10, symbol='circle', color='lightblue'))

# Update the hover template to only show the site name
fig.update_traces(hovertemplate="<b>%{hovertext}</b><extra></extra>", selector=dict(type='scattermapbox'))


fig.update_layout(
    map_style="carto-darkmatter",
    margin={"r":0,"t":0,"l":0,"b":0},
    clickmode='event+select' # Ensure clickmode is set to trigger selection events
)

app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])

app.layout = html.Div([
    html.Div([
        html.Div([
            html.H1("Rainfall Dashboard Ӏ Manawatū-Whanganui Region", style={'fontSize': '2em'}), # Adjusted font size and added the region name
            html.H2("Using OGC® WaterML 2.0 formatted data", style={'fontSize': '1.2em', 'margin-top': '0.5em'}) # Added subtitle with adjusted font size and margin
        ], style={'display': 'flex', 'flexDirection': 'column', 'alignItems': 'flex-start'}), # Use flexbox to align title and logo
        html.Div(id='output-container'),
        html.Div([
            html.H3("Select Date Range:", style={'fontSize': '1em'}), # Adjusted font size
            dcc.DatePickerRange(
                id='date-range-picker',
                start_date=(datetime.now() - timedelta(days=29)).date(), # Start 29 days ago to get 28 full days + today
                end_date=(datetime.now() - timedelta(days=1)).date(), # End yesterday
                display_format='DD/MM/YYYY'
            ),
        ], style={'margin-bottom': '20px'}), # Add some margin below the date picker
    ], style={'margin-left': '50px'}), # Add left margin to move content to the right
    dcc.Graph(
        id='site-map',
        figure=fig
    ),
    html.Div(id='rainfall-graphs-container', style={'display': 'flex', 'flexDirection': 'row', 'flex-wrap': 'wrap', 'width': '100%'})
])


@app.callback(
    [Output('output-container', 'children'),
     Output('rainfall-graphs-container', 'children'),
     Output('site-map', 'figure')], # Add output for the map figure
    Input('site-map', 'clickData'), # Ensure this ID matches the dcc.Graph ID
    Input('date-range-picker', 'start_date'),
    Input('date-range-picker', 'end_date'),
    State('site-map', 'figure') # Add state for the current map figure
)
def display_click_data_and_graph(clickData, start_date, end_date, current_fig):
    print("Callback function started") # Added print statement here
    print(f"clickData: {clickData}")
    print(f"start_date: {start_date}")
    print(f"end_date: {end_date}")

    if not clickData:
        print("No click data, returning default")
        # Reset selection when no site is clicked
        updated_fig = go.Figure(current_fig)
        updated_fig.update_traces(
            selector=dict(type='scattermapbox'),
            marker=dict(color='lightblue', size=10) # Reset to default style
        )
        return 'Click on a site on the map to see information.', None, updated_fig # Return updated figure

    site_name = clickData['points'][0]['hovertext']
    print(f"Clicked site: {site_name}")

    # Create a copy of the current figure to modify
    updated_fig = go.Figure(current_fig)
    print("Created a copy of the current figure")

    # Reset all markers to the default color (lightblue)
    updated_fig.update_traces(
        selector=dict(type='scattermapbox'),
        marker=dict(color='lightblue', size=10)
    )
    print("Reset all markers to lightblue")

    # Get the point index of the clicked site
    clicked_point_index = clickData['points'][0]['pointIndex']
    print(f"Clicked point index: {clicked_point_index}")

    # Update the marker of the clicked point to red and increase size
    updated_fig.update_traces(
        selector=dict(type='scattermapbox', pointdata=[{'pointIndex': clicked_point_index}]), # Select by point index
        marker=dict(color='red', size=12)
    )
    print(f"Updated marker color for point index {clicked_point_index} to red")


    if start_date and end_date:
        from_date_str = datetime.strptime(start_date, '%Y-%m-%d').strftime('%d/%m/%Y')
        to_date_str = datetime.strptime(end_date, '%Y-%m-%d').strftime('%d/%m/%Y')
        display_start_date = datetime.strptime(start_date, '%Y-%m-%d').strftime('%#d/%#m/%y') # Format for display
        display_end_date = datetime.strptime(end_date, '%Y-%m-%d').strftime('%#d/%#m/%y') # Format for display

    else:
        # Fallback to default 28 days if no date range is selected (shouldn't happen with initial values)
        today = datetime.now()
        yesterday = today - timedelta(days=1)
        twenty_eight_days_before_yesterday = yesterday - timedelta(days=28)
        from_date_str = twenty_eight_days_before_yesterday.strftime('%d/%m/%Y')
        to_date_str = yesterday.strftime('%d/%m/%Y')
        display_start_date = twenty_eight_days_before_yesterday.strftime('%#d/%#m/%y') # Format for display
        display_end_date = yesterday.strftime('%#d/%#m/%y') # Format for display


    selected_measurement = "rainfall"
    encoded_site = urllib.parse.quote(site_name)

    data_url = f"https://hilltopserver.horizons.govt.nz/data.hts?Service=Hilltop&Request=GetData&Site={encoded_site}&Measurement={selected_measurement}&From={from_date_str}&To={to_date_str}"
    print(f"Fetching data from URL: {data_url}")

    try:
        response = requests.get(data_url)
        response.raise_for_status()
        html_content = response.text
        soup = BeautifulSoup(html_content, 'xml')

        observations = soup.find_all('E')
        print(f"Found {len(observations)} observations")

        extracted_data = []
        for E in observations:
            t_tag = E.find('T')
            i1_tag = E.find('I1')
            if t_tag and i1_tag:
                try:
                    datetime_object = datetime.strptime(t_tag.text, "%Y-%m-%dT%H:%M:%S")
                    newdatetime = datetime_object.strftime("%Y-%m-%d %H:%M:%S")
                    extracted_data.append({
                        'T': newdatetime,
                        'I1': float(i1_tag.text)
                    })
                except ValueError as e:
                    print(f"Error parsing date or value: {e} for observation: {t_tag.text}, {i1_tag.text}")
                    continue

        if extracted_data:
            df_rainfall = pd.DataFrame(extracted_data)
            df_rainfall['T'] = pd.to_datetime(df_rainfall['T'])
            print(f"Successfully extracted {len(df_rainfall)} rainfall data points")

            df_daily_total = df_rainfall.groupby(df_rainfall['T'].dt.date)['I1'].sum().reset_index()
            df_daily_total.columns = ['Date', 'Rainfall']

            # Create a bar graph with gradient color
            fig_rainfall = px.bar(df_daily_total, x='Date', y='Rainfall',
                                  title=f'Daily Rainfall for {site_name}<br><sup>({display_start_date} to {display_end_date})</sup>', # Moved date range to subtitle
                                  labels={'Date': 'Date', 'Rainfall': 'Rainfall (mm)'},
                                  color='Rainfall',
                                  color_continuous_scale='Blues')

            # Add dark theme to the bar graph and adjust title font size
            fig_rainfall.update_layout(
                template="plotly_dark",
                xaxis=dict(tickformat='%d/%m/%y'), # Set date format for x-axis
                title=dict(
                    text=f'Daily Rainfall for {site_name}<br><sup>({display_start_date} to {display_end_date})</sup>',
                    font=dict(size=14) # Adjust title font size
                )
            )


            # Calculate daily rainfall totals for heatmap
            df_rainfall['date'] = df_rainfall['T'].dt.date
            df_rainfall['day_of_week'] = df_rainfall['T'].dt.day_name()
            df_rainfall['hour_of_day'] = df_rainfall['T'].dt.hour

            # Aggregate data for heatmap
            heatmap_data = df_rainfall.groupby(['day_of_week', 'hour_of_day'])['I1'].sum().unstack(fill_value=0)

            # Ensure consistent order for weekdays and hours
            weekdays_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
            heatmap_data = heatmap_data.reindex(weekdays_order, axis=0).fillna(0)
            hours_order = list(range(24))
            heatmap_data = heatmap_data.reindex(hours_order, axis=1).fillna(0)


            # Create the heatmap
            fig_heatmap = go.Figure(data=go.Heatmap(
                z=heatmap_data.values,
                x=heatmap_data.columns,
                y=heatmap_data.index,
                colorscale='Sunset',
                colorbar=dict(title='Total Rainfall (mm)'),
                hovertemplate = 'Hour: %{x}<br>Day: %{y}<br>Rainfall: %{z:.2f} mm<extra></extra>' # Added hovertemplate
            ))

            # Add dark theme to the heatmap and adjust title font size
            fig_heatmap.update_layout(
                title=dict(
                    text=f'Rainfall Heatmap per Hour per Weekday at {site_name}<br><sup>({display_start_date} to {display_end_date})</sup>',
                    font=dict(size=14) # Adjust title font size
                ),
                xaxis_title='Hour of Day',
                yaxis_title='Day of Week',
                xaxis = dict(
                    tickvals = list(range(24)),
                    ticktext = [str(hour) for hour in range(24)]
                ),
                template="plotly_dark" # Add dark theme to the heatmap
            )


            print("Successfully generated rainfall and heatmap figures")
            return (html.Div([
                html.P(f'You clicked on site: {site_name}'),
                html.P([
                    'Data for date range: ',
                    html.Span(f'{display_start_date} to {display_end_date}', style={'fontSize': '0.8em'}) # Adjusted font size for dates
                ])
            ]), html.Div([
                dcc.Graph(figure=fig_rainfall, style={'width': '50%', 'display': 'inline-block'}),
                dcc.Graph(figure=fig_heatmap, style={'width': '50%', 'display': 'inline-block'})
            ], style={'display': 'flex', 'flexDirection': 'row', 'width': '100%'}), updated_fig) # Return the updated figure

        else:
            print("No extracted data for rainfall")
            return html.Div([
                html.P(f'You clicked on site: {site_name}'),
                html.P(f'No rainfall data found for {site_name} in the selected date range.')
            ]), None, updated_fig # Return the updated figure

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return html.Div([
            html.P(f'You clicked on site: {site_name}'),
            html.P(f'Error fetching data: {e}')
        ]), None, updated_fig # Return the updated figure
    except Exception as e:
        print(f"An error occurred during data processing: {e}")
        return html.Div([
            html.P(f'You clicked on site: {site_name}'),
            html.P(f'An error occurred during data processing: {e}')
        ]), None, updated_fig # Return the updated figure

if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>

In [16]:
pip install session-info

In [17]:
import session_info
session_info.show()

/usr/local/lib/python3.12/dist-packages/session_info/main.py:213: UserWarning:

The '__version__' attribute is deprecated and will be removed in MarkupSafe 3.1. Use feature detection, or `importlib.metadata.version("markupsafe")`, instead.



In [18]:
# Use pip freeze to generate requirements.txt
!pip freeze > requirements.txt

print("requirements.txt file created successfully using pip freeze.")

requirements.txt file created successfully using pip freeze.
